In [2]:
import pandas as pd
root = 'data/'
lang = 'java'
ast = pd.read_pickle(root+lang+'/ast.pkl')

In [11]:
from utils import get_blocks_v1 as func
from gensim.models.word2vec import Word2Vec

word2vec = Word2Vec.load(root + lang + "/train/embedding/node_w2v_128").wv
max_token = word2vec.vectors.shape[0]
vocab = word2vec.vocab
#EMBEDDING_DIM = word2vec.vectors.shape[1]
#embeddings = np.zeros((MAX_TOKENS + 1, EMBEDDING_DIM), dtype = "float32")
#embeddings[:MAX_TOKENS] = word2vec.vectors

def tree_to_index(node):
    token = node.token
    result = [vocab[token].index if token in vocab else max_token]
    children = node.children
    for child in children:
        result.append(tree_to_index(child))
    return result

def trans2seq(r):
    blocks = []
    func(r, blocks)
    tree = []
    for b in blocks:
        btree = tree_to_index(b)
        tree.append(btree)
    return tree

In [12]:
ast = pd.DataFrame(ast, copy=False)
ast['code'] = ast['code'].apply(trans2seq)

In [13]:
ast

,id,code
0,74,"[[25, [20, [75]], [181], [19, [4, [16]], [158]..."
1,661,"[[25, [20, [33]], [1107], [19, [4, [34]], [158..."
2,1362,"[[25, [20, [33], [75]], [15, [22]], [2957], [1..."
3,1363,"[[25, [20, [33], [75]], [15, [22]], [2957], [1..."
4,2450,"[[25, [20, [33]], [784], [19, [4, [16]], [335]..."
...,...,...
59613,23653940,"[[327, [2957], [19, [4, [72]], [2957]], [19, [..."
59614,23655348,"[[25, [20, [33]], [15, [76]], [2957], [19, [4,..."
59615,23672349,"[[25, [20, [33], [75]], [15, [41]], [2957], [1..."
59616,23672350,"[[25, [20, [33], [75]], [15, [41]], [2957], [1..."


In [15]:
ast.to_pickle(root+lang+'/ast_embedded.pkl')

In [1]:
import numpy as np
import pandas as pd
from model import BatchProgramCC
from gensim.models.word2vec import Word2Vec
import torch

def get_device():
    if torch.cuda.is_available():
        if torch.cuda.get_device_name(0) == 'GeForce GT 730':
            device = 'cpu'
        else:
            device = 'cuda'
    else:
        device = 'cpu'
    return torch.device(device)

device = get_device()


root = 'data/'
lang = 'java'
categories = 1
if lang == 'java':
    categories = 5

ast_embedded = pd.read_pickle(root+lang+'/ast_embedded.pkl')

word2vec = Word2Vec.load(root+lang+"/train/embedding/node_w2v_128").wv
MAX_TOKENS = word2vec.vectors.shape[0]
EMBEDDING_DIM = word2vec.vectors.shape[1]
#embeddings = np.zeros((MAX_TOKENS + 1, EMBEDDING_DIM), dtype="float32")
#embeddings[:MAX_TOKENS] = word2vec.vectors

C:\Users\A1\anaconda3\envs\pytorch\lib\site-packages\torch\cuda\__init__.py:102: UserWarning: 
    Found GPU0 GeForce GT 730 which is of cuda capability 3.0.
    PyTorch no longer supports this GPU because it is too old.
    The minimum cuda capability that we support is 3.5.
    
  warnings.warn(old_gpu_warn % (d, name, major, capability[1]))
C:\Users\A1\anaconda3\envs\pytorch\lib\site-packages\torch\cuda\__init__.py:125: UserWarning: 
GeForce GT 730 with CUDA capability sm_30 is not compatible with the current PyTorch installation.
The current PyTorch install supports CUDA capabilities sm_37 sm_50 sm_60 sm_61 sm_70 sm_75 compute_37.
If you want to use the GeForce GT 730 GPU with PyTorch, please check the instructions at https://pytorch.org/get-started/locally/

  warnings.warn(incompatible_device_warn.format(device_name, capability, " ".join(arch_list), device_name))


In [2]:
import math

class dataloader():
    def __init__(self, dataset, batch_size):
        self.dataset = dataset
        self.batch_size = batch_size
        self.idx = 0
        self.max_idx = len(dataset)
        self.len = math.ceil(len(self.dataset) / self.batch_size)
        
    def __call__(self):
        pass
    
    def __len__(self):
        return self.len
    
    def __iter__(self):
        self.idx = 0
        return self
    
    def __next__(self):
        if self.idx >= self.max_idx:
            raise StopIteration
        tmp = self.dataset.iloc[self.idx: self.idx+self.batch_size]
        x = []
        for _, item in tmp.iterrows():
            x.append(item['code'])
        self.idx += self.batch_size
        return x
    


In [3]:
HIDDEN_DIM = 100
ENCODE_DIM = 128
LABELS = 1
BATCH_SIZE = 128
type_str = ['T1', 'T2', 'ST3', 'MT3', 'T4']
model_file = [' ',
              ' ',
              ' ',
              ' ',
              '20200927_075407 java ccd model for T4 epoch_13 val.pt']
save_dir = '---- java ccd model/'


model = BatchProgramCC(EMBEDDING_DIM,
                       HIDDEN_DIM,
                       MAX_TOKENS+1,
                       ENCODE_DIM,
                       LABELS,
                       BATCH_SIZE,
                       device).to(device)

data = dataloader(ast_embedded, BATCH_SIZE)

In [4]:
T = 5
param = torch.load(save_dir + type_str[T-1] + '/' + model_file[T-1])['param']
model.load_state_dict(param)
model.eval()

BatchProgramCC(
  (encoder): BatchTreeEncoder(
    (embedding): Embedding(2958, 128)
    (W_c): Linear(in_features=128, out_features=128, bias=True)
  )
  (bigru): GRU(128, 100, batch_first=True, bidirectional=True)
)

In [5]:
with open(root+'Xmeans dataset/'+type_str[T-1]+'.txt', 'w') as f:
    with torch.no_grad():
        for x in data:
            model.batch_size = len(x)
            model.hidden = model.init_hidden()
            encode = model.encode(x).cpu().numpy()
            buffer = []
            for s in encode:
                i = [str(i) for i in s]
                i = ', '.join(i)
                buffer.append(i)
            buffer = '\n'.join(buffer)
            f.write(buffer+'\n')
            #break

In [15]:
with open(root+'Xmeans dataset/'+type_str[T-1]+'_1000.txt', 'w') as f_w:
    with open(root+'Xmeans dataset/'+type_str[T-1]+'.txt', 'r') as f_r:
        for line in f_r:
            temp=line.strip('\n').split(', ')
            temp = [str(float(i)*1000) for i in temp]
            f_w.write(', '.join(temp)+'\n')
            #break
            